In [3]:
import numpy as np
import pandas as pd
from scipy.spatial import distance
import cv2
import math

In [4]:
def eye_aspect_ratio(eye):
	A = distance.euclidean(eye[1], eye[5])
	B = distance.euclidean(eye[2], eye[4])
	C = distance.euclidean(eye[0], eye[3])
	ear = (A + B) / (2.0 * C)
	return ear
def mouth_aspect_ratio(mouth):
    A = distance.euclidean(mouth[14], mouth[18])
    C = distance.euclidean(mouth[12], mouth[16])
    mar = (A ) / (C)
    return mar
def circularity(eye):
    A = distance.euclidean(eye[1], eye[4])
    radius  = A/2.0
    Area = math.pi * (radius ** 2)
    p = 0
    p += distance.euclidean(eye[0], eye[1])
    p += distance.euclidean(eye[1], eye[2])
    p += distance.euclidean(eye[2], eye[3])
    p += distance.euclidean(eye[3], eye[4])
    p += distance.euclidean(eye[4], eye[5])
    p += distance.euclidean(eye[5], eye[0])
    return 4 * math.pi * Area /(p**2)
def mouth_over_eye(eye):
	ear = eye_aspect_ratio(eye)
	mar = mouth_aspect_ratio(eye)
	mouth_eye = mar/ear
	return mouth_eye

In [5]:
landmarks=np.load('landmarks.npy')
data=np.load('data.npy')
labels=np.load('labels.npy')
respondent=np.load('respondents.npy')

In [6]:
len(landmarks)

68

In [7]:
ear=[]
mar=[]
moe=[]
circu=[]


In [6]:
landmarks[67]

array([541, 743])

In [7]:
for i in data:
    eye=i[36:67]
    ear.append(eye_aspect_ratio(eye))
    mar.append(mouth_aspect_ratio(eye))
    moe.append(mouth_over_eye(eye))
    circu.append(circularity(eye))

In [15]:
len(ear)

1800

In [8]:
data= pd.DataFrame(list(zip(respondent,moe,mar,ear,circu,labels)),columns=['Respondents','MOE','MAR','EAR','Circularity','Labels'])

In [18]:
data.to_csv('Main.csv')

# normalisation

In [9]:
alert=data[data['Labels']==0]

In [10]:
alert1=alert.iloc[0::300,:]
alert2=alert.iloc[1::300,:]
alert3=alert.iloc[2::300,:]

In [11]:
alert_all=pd.concat([alert1,alert2,alert3])
alert_all=alert_all.sort_index()

In [12]:
alert_mean=alert_all.groupby("Respondents")['MOE','MAR','EAR','Circularity'].mean()
alert_std=alert_all.groupby("Respondents")['MOE','MAR','EAR','Circularity'].std()


In [13]:
def mean_EAR(respondent):
    return alert_mean.loc[respondent]["EAR"]

def mean_MAR(respondent):
    return alert_mean.loc[respondent]["MAR"]

def mean_Circularity(respondent):
    return alert_mean.loc[respondent]["Circularity"]

def mean_MOE(respondent):
    return alert_mean.loc[respondent]["MOE"]


def std_EAR(respondent):
    return alert_std.loc[respondent]["EAR"]

def std_MAR(respondent):
    return alert_std.loc[respondent]["MAR"]

def std_Circularity(respondent):
    return alert_std.loc[respondent]["Circularity"]

def std_MOE(respondent):
    return alert_std.loc[respondent]["MOE"]

In [17]:
data=pd.read_csv('Main.csv')

In [18]:
data['mean_MOE']=data['Respondents'].apply(mean_MOE)
data['mean_EAR']=data['Respondents'].apply(mean_EAR)
data['mean_MAR']=data['Respondents'].apply(mean_MAR)
data['mean_Circularity']=data['Respondents'].apply(mean_Circularity)

In [19]:
data['std_MOE']=data['Respondents'].apply(std_MOE)
data['std_EAR']=data['Respondents'].apply(std_EAR)
data['std_MAR']=data['Respondents'].apply(std_MAR)
data['std_Circularity']=data['Respondents'].apply(std_Circularity)

In [20]:
data['Normalised_EAR']=(data['EAR']-data['mean_EAR'])/data['std_EAR']
data['Normalised_MAR']=(data["MAR"]-data['mean_MAR'])/data['std_MAR']
data['Normalised_MOE']=(data['MOE']-data['mean_MOE'])/data['std_MOE']
data['Normalised_Circularity']=(data['Circularity']-data['mean_Circularity'])/data['std_Circularity']

In [21]:
#final_data=data.drop(columns=['MOE','MAR','EAR','Circularity'])

In [23]:
final_data=data.drop(columns=['mean_MAR','mean_MOE','mean_Circularity','mean_EAR','Unnamed: 0','std_MOE','std_MAR','std_EAR','std_Circularity'])

In [24]:
final_data.columns

Index(['Respondents', 'MOE', 'MAR', 'EAR', 'Circularity', 'Labels',
       'Normalised_EAR', 'Normalised_MAR', 'Normalised_MOE',
       'Normalised_Circularity'],
      dtype='object')

In [25]:
final_data.to_csv('final2.csv')

In [26]:
final_data.shape

(1800, 10)